In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Effective field for chiral TBG model

Goal: use the same transformation as Morales-Durán et al. (https://arxiv.org/pdf/2308.03143.pdf) used to obtain Dirac Landau levels for a twisted bilayer graphene model.